In [1]:
import datetime
import random

import numpy as np
import yaml
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import csv

from utils.data_reading.features_extractor import STFTFeaturesExtractor, RelativeDWTFeaturesExtractor, RawDataFeaturesExtractor
from utils.data_reading.sound_file_manager import WavFilesManager
from utils.misc.misc import make_directory

## Parameters

In [2]:
yaml_path = "/home/plerolland/Bureau/OHASISBIO_dataset/data/dataset.yaml"

output_path = "/media/plerolland/LaBoite/PublicData/training/raw"

# duration taken before and after the event for the output
delta_output = datetime.timedelta(seconds=50)

# type of data that we want
#features_extractor = STFTFeaturesExtractor(None, vmin=-35, vmax=140)  # spectrograms generator
#features_extractor = RelativeDWTFeaturesExtractor(None)  # DWT features generator
features_extractor = RawDataFeaturesExtractor(None)  # DWT features generator

random_offset_multiplicator = 1  # set to 0 if we want to center events

SEED = 0

## Initialization

In [3]:
random.seed(SEED)

with open(yaml_path, "r") as f:
    params = yaml.load(f, Loader=yaml.BaseLoader)["train_dataset"]
prefix = params["root_dir"]
stations = list(params["stations"].keys())

dataset_csv_path = f"{output_path}/dataset.csv"

positives = {h : [] for h in stations}
negatives = {h : [] for h in stations}
managers = {h : WavFilesManager(f"{prefix}/{h}") for h in stations}

make_directory(output_path)
make_directory(f"{output_path}/positives")
make_directory(f"{output_path}/negatives")

csv_data = []

## Data loading

In [4]:
with open(f"{prefix}/dataset.csv") as f:
    csv_reader = csv.reader(f, delimiter=",")
    next(csv_reader) # skip first line which contains column names
    lines = list(csv_reader)
for line in lines:
    station = line[0]
    date = datetime.datetime.strptime(line[2], "%Y%m%d_%H%M%S")
    if line[1] == "positive":
        positives[station].append(date)
    else:
        negatives[station].append(date)

## Dataset creation

### Positives

In [5]:
dates = {station: [] for station in stations}
segments_to_save = {station: [] for station in stations}

# choose segments (with a random offset) for the positives
for station, d in positives.items():
    d = np.sort(d)
    for i in range(len(d)):
        dates[station].append([])
        offset = (random.random()-0.5) * 1.9 * delta_output * random_offset_multiplicator
        segments_to_save[station].append((d[i] + offset - delta_output, d[i] + offset + delta_output))
        
        j=i # look for events before this one that may appear in the window
        while j>=0 and abs(d[j] - (d[i] + offset)) <= delta_output:
            event_pos = d[j] - (d[i] + offset)
            dates[station][-1].append(f"{event_pos.total_seconds():.1f}")
            j -= 1
        j=i+1 # look for events after this one that may appear in the window
        while j<len(d) and abs(d[j] - (d[i] + offset)) <= delta_output:
            event_pos = d[j] - (d[i] + offset)
            dates[station][-1].append(f"{event_pos.total_seconds():.1f}")
            j += 1

# compute and save spectrogram
for station in tqdm(positives.keys()):
    features_extractor.manager = managers[station]
    starts, ends = zip(*segments_to_save[station])
    path_prefix = f'{output_path}/positives/{station}'
    
    if isinstance(features_extractor, RelativeDWTFeaturesExtractor):
        # make only 1 file for the station because data representation is small enough
        path = f'{path_prefix}.{features_extractor.EXTENSION}'
        features_extractor.save_features_batch_single_file(starts, ends, path)
        csv_data.extend([[path, str(i), "positive"] + dates[station][i] for i in range(len(starts))])
    else:
        # make 1 folder per station
        make_directory(path_prefix)
        d = positives[station]
        paths = [f'{path_prefix}/{d[i].strftime("%Y%m%d_%H%M%S")}.{features_extractor.EXTENSION}' for i in range(len(d))]
        csv_data.extend([[paths[i], "positive"] + dates[station][i] for i in range(len(starts))])
        features_extractor.save_features_batch(starts, ends, paths)

100%|██████████| 9/9 [01:42<00:00, 11.36s/it]


### Negatives

In [6]:
for station in tqdm(negatives.keys()):
    features_extractor.manager = managers[station]
    starts, ends = zip(*[(negatives[station][i] - delta_output, negatives[station][i] + delta_output) for i in range(len(negatives[station]))])
    path_prefix = f'{output_path}/negatives/{station}'
    
    if isinstance(features_extractor, RelativeDWTFeaturesExtractor):
        # make only 1 file for the station because data representation is small enough
        path = f'{path_prefix}.{features_extractor.EXTENSION}'
        features_extractor.save_features_batch_single_file(starts, ends, path)
        csv_data.extend([[path, str(i), "negative"] for i in range(len(starts))])
    else:
        # make 1 folder per station
        make_directory(path_prefix)
        d = negatives[station]
        paths = [f'{path_prefix}/{d[i].strftime("%Y%m%d_%H%M%S")}.{features_extractor.EXTENSION}' for i in range(len(d))]
        csv_data.extend([[paths[i], "negative"] for i in range(len(starts))])
        features_extractor.save_features_batch(starts, ends, paths)

100%|██████████| 9/9 [04:58<00:00, 33.17s/it]


In [7]:
with open(dataset_csv_path, "w") as f:
    csv.writer(f).writerows(csv_data)